В этом задании мы должны научить машину понимать тексты. В файле recs.txt находятся отзывы о разных фильмах. Наша модель машинного обучения должна проанализировать отзыв и ответить на простой вопрос: понравился ли пользователю просмотренный фильм.

Подключите необходимые для работы пакеты: pandas as pd, numpy as np, nltk, re, все функции из sklearn

In [141]:
import pandas as pd
import numpy as np
import nltk
import sklearn
import re

Увеличьте ширину выводимой информации в DataFrame (на это понадобится для показа текстов отзывав):
pd.set_option('max_colwidth',600)

In [142]:
pd.set_option('max_colwidth',600)

Прочтите данные из файла recs.txt, в качестве разделителя (параметр sep) там выступает строка "\\-\\-\\|\\|\\|\\-\\-", кодировка (параметр encoding) - 'utf-8'.

Выведите три первых строки таблицы. Какое отношение (Sentiment) у авторов отзывов к просмотренным фильмам?

In [143]:
df = pd.read_csv("recs.txt",sep="\-\-\|\|\|\-\-",encoding = 'utf-8')
df.head(3)

C:\Users\google\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,Sentiment,URL,Text
0,-1,kinopoisk.ru/user/72383/comment/846161/,"На самых первых титрах у меня появилось абсолютно твёрдое и непоколебимое ощущение, что я смотрю совершено детский фильм, в духе «Детей шпионов», иначе как объяснить наличие в начальных титрах техно-музыки и огромнейшего, просто колоссальнейшего наличия компьютерной графики, уровня приставок фирмы Sega. Если кто из вас по детству удосужился сыграть в видео-игру по мотивам мультфильма «Король-лев», адаптированную специально для этого платформера, тот прекрасно меня поймёт. Для полноты картины не хватает только мангуста Димона, который выдал бы в камеру памятное «It starts!». Хотя в данном ф..."
1,1,kinopoisk.ru/user/697175/comment/805374/,"Вы знаете, неожиданно хороший фильм. Я не большой любитель фильмов такого жанра, но этот мне очень понравился, совсем не ожидала. Придраться, как бы сильно ни хотелось это сделать, не к чему: Сюжет необычный, непривычный, оригинальный и оторваться от просмотра практически невозможно. Весь фильм проходит в движении: вечно что-то случается, открываются какие-то тайны, всевозможные неожиданные повороты и напряженные моменты. Как я уже сказала, если начнете смотреть этот фильм, остановиться уже не сможете. Я, к примеру, просто не смогла оторваться, практически прилипла к экрану. Персонажи - за..."
2,1,kinopoisk.ru/user/1335875/comment/2079675/,"Да, этот фильм такой, что всё, что там покажут, можно было угадать заочно. Достаточно было посмотреть на постер и прочесть название. Однако решил ввязаться в просмотр. В принципе, могу сказать, что не жалею времени, потраченного на это кино. Хоть это и не выдающаяся картина, было увлекательно посмотреть на приятных глазу актёров, а временами даже смешно. Это мелодрама с элементами комедии. Или любовная комедия. Или комедия с девичьим уклоном. Однако, думаю, и парням будет посмотреть что. В дебюте так вообще главные герои занимаются сексом везде и в каждом месте, словно слетевшие с катушек...."


Ну очевидно что первому фильм не понравился и 	Sentiment=-1, всем остальным ок) 

Чтобы преобразовать тексты в наборы признаков методом bag-of-words мы должны сначала профильтровать текст, оставив только значащие слова. Скачайте из корпуса набор незначащих слов для фильтрации командой nltk.download()  (в появившемся окне вкладка Corpora, пункт stopwords). 

In [144]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\google\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Импортируйте список незначащих слов командами 
```python
from nltk.corpus import stopwords
stopwords = set(stopwords.words("russian"))
```
и выведите его на экран.

In [145]:
from nltk.corpus import stopwords
stopwords = set(stopwords.words("russian"))

Определите функцию фильтрации текста:
```python
def review_to_words( review_text ):
    review_text = review_text.replace('ё','е')
    review_text = review_text.replace('Ё','Е')
    # 1. Удаляем все, кроме букв
    letters_only = re.sub("[^а-яА-Я]", " ", review_text) 
    # 2. Делаем все буквы строчными и создаем массив слов
    words = letters_only.lower().split()                             
    # 3. Удаляем незначащие слова
    meaningful_words = [w for w in words if not w in stopwords]
    # 4. Формируем текст, объединяя слова через пробел
    return( " ".join( meaningful_words ))
```

In [146]:
def review_to_words( review_text ):
    review_text = review_text.replace('ё','е')
    review_text = review_text.replace('Ё','Е')
    # 1. Удаляем все, кроме букв
    letters_only = re.sub("[^а-яА-Я]", " ", review_text) 
    # 2. Делаем все буквы строчными и создаем массив слов
    words = letters_only.lower().split()                             
    # 3. Удаляем незначащие слова
    meaningful_words = [w for w in words if not w in stopwords]
    # 4. Формируем текст, объединяя слова через пробел
    return( " ".join( meaningful_words ))

Скопируйте базу данных отзывов с помощью метода copy() в новую переменную и примените к отзывам функцию фильтрации: .apply(lambda s: review_to_words(s)).

Выведите на экран первые три строчки преобразованной базы данных

In [147]:
df_copy = df.copy()
df_copy['Text'] = df_copy['Text'].apply(lambda s: review_to_words(s))
df_copy.head(3)

,Sentiment,URL,Text
0,-1,kinopoisk.ru/user/72383/comment/846161/,самых первых титрах появилось абсолютно твердое непоколебимое ощущение смотрю совершено детский фильм духе детей шпионов иначе объяснить наличие начальных титрах техно музыки огромнейшего просто колоссальнейшего наличия компьютерной графики уровня приставок фирмы детству удосужился сыграть видео игру мотивам мультфильма король лев адаптированную специально платформера прекрасно поймет полноты картины хватает мангуста димона который выдал камеру памятное хотя данном фильме фильму подошло сидишь неволей думаешь попал фильм адресованный поколению ясельной группы детского сада сюжет большему с...
1,1,kinopoisk.ru/user/697175/comment/805374/,знаете неожиданно хороший фильм большой любитель фильмов такого жанра очень понравился ожидала придраться сильно хотелось это сделать чему сюжет необычный непривычный оригинальный оторваться просмотра практически невозможно весь фильм проходит движении вечно случается открываются какие тайны всевозможные неожиданные повороты напряженные моменты сказала начнете смотреть фильм остановиться сможете примеру просто смогла оторваться практически прилипла экрану персонажи замечательные помощи актеров которых подобрали ким бейсингер просто шикарна хотя видеть роли жертвы очень непривычно равно здо...
2,1,kinopoisk.ru/user/1335875/comment/2079675/,фильм покажут угадать заочно достаточно посмотреть постер прочесть название однако решил ввязаться просмотр принципе могу сказать жалею времени потраченного это кино это выдающаяся картина увлекательно посмотреть приятных глазу актеров временами смешно это мелодрама элементами комедии любовная комедия комедия девичьим уклоном однако думаю парням посмотреть дебюте вообще главные герои занимаются сексом везде каждом месте словно слетевшие катушек эштон кутчер актер который никак профессионал сформироваться моем сознании часто картины номинируются золотую малину эффект бабочки всякий заставля...


Запустите преобразователь текстового признака в «bag of words» с помощью класса CountVectorizer библиотеки sklearn.feature_extraction.text. 

В конструкторе класса укажите параметры: analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 50000, binary = True

Вызовите метод обучения (построения словаря) и преобразования текстов: fit_transform. Результат присвойте новой переменной bagOfWords.

Сохраните порядок слов в словаре (метод .get_feature_names()) в переменную vocabulary.

In [164]:
bag_of_worlds = sklearn.feature_extraction.text.CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, 
                                               stop_words = None, max_features = 50000, binary = True)

bagOfWords = bag_of_worlds.fit_transform(df_copy['Text'])
df_array = df_copy.copy()

df_array['Text'] = np.array(bagOfWords)
vocabulary = bag_of_worlds.get_feature_names_out()

Теперь в наших матрицах объектов-признаков 50 тысяч столбцов и в сумме 100 тысяч строк. Если 1 элемент занимает 1 байт, то суммарный объем будет 5 Гб. Проверьте по диспетчеру задач (системный монитор в Linux), сколько занимает памяти ваша программа и напишите в комментариях. Все ли согласуется?

In [149]:
#2500Mb я думаю оно как-то хитро кэшируется или данные в swap находятся 

Разбейте выборку bagOfWords на две части: обучающую (строки до 50000) и проверочную (строки >= 50000). Назовите их bagOfWordsTrain и bagOfWordsTest

In [150]:
bagOfWordsTrain = bagOfWords[:50000]
bagOfWordsTest = bagOfWords[50000:]
bagOfWordsTrain_y =df_array['Sentiment'][:50000]
bagOfWordsTest_y =df_array['Sentiment'][50000:]

Сконструируйте машину опорных векторов svm.LinearSVC с параметрами C=1, fit_intercept=False и присвойте ее переменной SVM.

Натренируйте ее на обучающей выборке и с помощью метода .score(x,y) вычислите процент правильных предсказаний для тестовой выборки.

In [151]:
SVM = sklearn.svm.LinearSVC(C=0.7, fit_intercept=False)
SVM.fit(bagOfWordsTrain, bagOfWordsTrain_y)
SVM.score(bagOfWordsTest, bagOfWordsTest_y)

0.91032

Подберите важность суммы нарушений C, чтобы SVM работала лучше всего

Выведите на экран 20 слов, которые больше всего голосуют за позитивность отзыва и 20 — за негативность вместе с соответствующими им коэффициентами натренированной линейной функции:
```python
ind = np.argsort(SVM.coef_[0])
for i in range(20):
    print(SVM.coef_[0][ind[i]], vocabulary[ind[i]])
print("\n")
for i in range(20):
    print(SVM.coef_[0][ind[-i-1]], vocabulary[ind[-i-1]]) 
```

In [152]:
ind = np.argsort(SVM.coef_[0])
for i in range(20):
    print(SVM.coef_[0][ind[i]], vocabulary[ind[i]])
print("\n")
for i in range(20):
    print(SVM.coef_[0][ind[-i-1]], vocabulary[ind[-i-1]]) 

-0.5602024666221733 разочарование
-0.5364105211918703 провал
-0.4707754778192382 ужасен
-0.47017784239919147 рекламе
-0.46905640421943895 надеялся
-0.4677213262789189 разочарована
-0.45891259663578987 скучная
-0.45857207394577304 непонятными
-0.44752283012135907 отсутствует
-0.44380250124550774 тети
-0.44305153085028764 увы
-0.43304797109063625 передачи
-0.42928001191440407 досмотреть
-0.4270841921701173 советовать
-0.4147868384521107 скучный
-0.40733080676569317 коротким
-0.40658093250733074 вяло
-0.40316225708706993 затянутый
-0.39458627462495877 отвратительно
-0.39299928311251725 плюсы


0.4337091674945392 отпускает
0.4290634027415527 придраться
0.42250788848950677 отличный
0.4202547354066424 плавно
0.4054365585307551 шикарен
0.40272396324150755 счастья
0.3985355841032722 прекрасен
0.3772626844142897 потрясающе
0.3768078824420075 великолепно
0.37667621731590883 одновременно
0.37090438008816323 оторваться
0.366176473871368 современному
0.3602278061966411 великолепные
0.34938467739400

Какой коэффициент соответствует словам: 'критика', 'странно', 'понимаю', 'деньги', 'даже' и еще 5 ваших слов по выбору. За что они голосуют: позитивность или негативность отзыва?

In [154]:
word = ['критика', 'странно', 'понимаю', 'деньги', 'хороший', 'плохой', 'спал', 'устал', 'куча']
for i in word:
    print(SVM.coef_[0][np.where(vocabulary == i)[0]][0], i)


0.021576324554399888 критика
0.15841668417412388 странно
-0.24453945199309393 понимаю
-0.11088210235128006 деньги
0.2671852411154425 хороший
-0.18548524792932977 плохой
-0.09427011751533823 спал
0.10985302314447117 устал
0.027842489642851402 куча


Вычислите выступы, найдите и выведете на экран отзывы с минимальным и максимальным выступом, величину выступа, величину предсказания и сентиментность отзыва. Объясните в комментариях, почему найденный отзыв имеет такой маленький выступ, а другой найденный — такой большой:
```python
prediction = SVM.decision_function(bagOfWordsTest)
margins = np.multiply(prediction, data.loc[50000:]['Sentiment'].as_matrix())
ind = np.argsort(margins)
i = ind[-1] + 50000
print('Величина выступа = ', margins[ind[-1]])
print('Величина предсказания = ', prediction[ind[-1]])
print('Сентиментность отзыва = ', data.loc[i,'Sentiment'])
print('\n')
print(data.loc[i,'Text'])
print("\n")
i = ind[0] + 50000
print('Величина выступа = ', margins[ind[0]])
print('Величина предсказания = ', prediction[ind[0]])
print('Сентиментность отзыва = ', data.loc[i,'Sentiment'])
print('\n')
print(data.loc[i,'Text'])
```

Для объяснения отступов вам возможно пригодится код, который печатает коэффициенты w<sub>i</sub> линейной функции, значения соответствующих признаков и соответствующие слова. И все это делается в порядке убывания модуля произведения w<sub>i</sub>x<sub>i</sub>:
```python
k = ind[0]
ind2 = np.argsort(np.abs(np.multiply(SVM.coef_[0], bagOfWordsTest[k].todense()))).reshape(-1,1)
s = 0
for i in range((bagOfWordsTest[k]!=0).sum()):
    ii = ind2[-i-1].item()
    print('wi='+str(SVM.coef_[0][ii]), 'xi='+str(bagOfWordsTest[k,ii]),  vocabulary[ii])
    s += SVM.coef_[0][ii]*bagOfWordsTest[k,ii]
print('Проверка:',s) 
```

In [155]:
prediction = SVM.decision_function(bagOfWordsTest)
margins = np.multiply(prediction, df.loc[50000:]['Sentiment'].to_numpy())
ind = np.argsort(margins)
i = ind[-1] + 50000
print('Величина выступа = ', margins[ind[-1]])
print('Величина предсказания = ', prediction[ind[-1]])
print('Сентиментность отзыва = ', df.loc[i,'Sentiment'])
print('\n')
print(df.loc[i,'Text'])
print("\n")
i = ind[0] + 50000
print('Величина выступа = ', margins[ind[0]])
print('Величина предсказания = ', prediction[ind[0]])
print('Сентиментность отзыва = ', df.loc[i,'Sentiment'])
print('\n')
print(df.loc[i,'Text'])

Величина выступа =  12.899802966249753
Величина предсказания =  12.899802966249753
Сентиментность отзыва =  1


Иногда, когда смотришь из окна дома на ночной город с его бесконечными дорогами, освещающими пути и переходы фонарями, мигающими светофорами и рекламными вывесками, сверкающими неоновыми огнями, то мысленно восторгаешься красотой, загадочностью и таинственностью открывающегося вида. Ведь ты абсолютно не знаешь куда спешат те редкие в это время суток прохожие - кто-то возвращается из кино, кто-то припозднился на работе и усталыми шагами следует домой, кто-то идет на свидание заранее предвкушая радость встречи, а кто-то просто ловит случайное такси, которое этой ночью совершит наверное свой самый страшный маршрут. А что если очередной пассажир севший на заднее сидение машины, профессиональный киллер, который собирается осуществить за определенный промежуток времени целых пять убийств? Он очень умен, хладнокровен и бесконечно хитер, и следует заранее посочувствовать водителю так

In [156]:
k = ind[0]
ind2 = np.argsort(np.abs(np.multiply(SVM.coef_[0], bagOfWordsTest[k].todense()))).reshape(-1,1)
s = 0
for i in range((bagOfWordsTest[k]!=0).sum()):
    ii = ind2[-i-1].item()
    print('wi='+str(SVM.coef_[0][ii]), 'xi='+str(bagOfWordsTest[k,ii]),  vocabulary[ii])
    s += SVM.coef_[0][ii]*bagOfWordsTest[k,ii]
print('Проверка:',s) 

wi=0.3772626844142897 xi=1 потрясающе
wi=0.366176473871368 xi=1 современному
wi=-0.3288492058739642 xi=1 позволения
wi=-0.2941408302613557 xi=1 неестественно
wi=0.27878898338057445 xi=1 остроумные
wi=0.27020434456469467 xi=1 аспект
wi=0.269316932905493 xi=1 фильм
wi=0.2650270013729449 xi=1 слегка
wi=0.2541540643897264 xi=1 обязательно
wi=-0.2444301524730296 xi=1 разговоры
wi=0.24343130607472696 xi=1 действительность
wi=0.2422727729919302 xi=1 закона
wi=0.23687298370691157 xi=1 сегодняшнего
wi=-0.23216744703659653 xi=1 вывод
wi=0.2317926838435085 xi=1 неоднозначный
wi=-0.23081845704607087 xi=1 абсолютно
wi=-0.22666114253466702 xi=1 комикса
wi=0.2251523561600138 xi=1 научно
wi=0.22438326644613504 xi=1 силе
wi=0.223294126432331 xi=1 трилогии
wi=-0.2214844398573213 xi=1 очевидный
wi=0.22132554017215084 xi=1 свежей
wi=0.21968701325847217 xi=1 родители
wi=-0.2192735507238846 xi=1 сожалению
wi=-0.21762410918395064 xi=1 полке
wi=0.212593884783738 xi=1 трилогия
wi=-0.21250228586160544 xi=1 фило

Нормализуйте признаки перед разбиением выборки на Train и Test так, чтобы вместо 0/1 стояли частоты употребления слов, деленные на длину отзыва. Подберите наилучшую константу C и опишите, как изменятся отзывы с минимальным и максимальным выступом? Что лучше: не нормализованные или нормализованные признаки? Почему?

Нужно установить binary = False в CountVectorizer и добавить после fit_transform
```python
bagOfWords = preprocessing.normalize(bagOfWords, norm='l1')
```

Через веса слов считается выступ, а негативные слова имеют отрицательный вес, а положительный положительный, и в зависимости от характера текста, выступ маленький или же большой + влияет частота втречи, а тексты большие поскольку не нормализованные, а обычно люди пишут либо много хорошего либо много плохого)))

In [157]:
bagOfWords = sklearn.feature_extraction.text.CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, 
                                               stop_words = None, max_features = 50000, binary = False)
bagOfWords = bagOfWords.fit_transform(df_copy['Text'])
bagOfWords = sklearn.preprocessing.normalize(bagOfWords, norm='l1')


bagOfWordsTrain = bagOfWords[:50000]
bagOfWordsTest = bagOfWords[50000:]
bagOfWordsTrain_y =df_array['Sentiment'][:50000]
bagOfWordsTest_y =df_array['Sentiment'][50000:]




In [162]:
SVM = sklearn.svm.LinearSVC(C=150, fit_intercept=False)
SVM.fit(bagOfWordsTrain, bagOfWordsTrain_y)
SVM.score(bagOfWordsTest, bagOfWordsTest_y)

0.92682

In [163]:
prediction = SVM.decision_function(bagOfWordsTest)
margins = np.multiply(prediction, df.loc[50000:]['Sentiment'].to_numpy())
ind = np.argsort(margins)
i = ind[-1] + 50000
print('Величина выступа = ', margins[ind[-1]])
print('Величина предсказания = ', prediction[ind[-1]])
print('Сентиментность отзыва = ', df.loc[i,'Sentiment'])
print('\n')
print(df.loc[i,'Text'])
print("\n")
i = ind[0] + 50000
print('Величина выступа = ', margins[ind[0]])
print('Величина предсказания = ', prediction[ind[0]])
print('Сентиментность отзыва = ', df.loc[i,'Sentiment'])
print('\n')
print(df.loc[i,'Text'])

Величина выступа =  8.630282058642583
Величина предсказания =  8.630282058642583
Сентиментность отзыва =  1


Действительно, интересный молодежный фильм, немного наивный, но приятный для просмотра, ну, и грустный местами. Он на все 100% дает то, что можно ожидать от таких фильмов, и может даже немного больше! .


Величина выступа =  -4.66017362485359
Величина предсказания =  -4.66017362485359
Сентиментность отзыва =  1


«Придурки» продолжают свое победное шествие по большим экранах. Их игрища конечно на любителя, но их второе пришествие меня разочаровала по сравнению с первой частью. Иногда смешно, иногда очень смешно, то чаще противно, тупо и ужасно и.... Вывод - поржать можно, но от употребления пищи при этом следует отказаться, а то постигнет разочарование...


После нормализации нащи коментраии с выступами стали маленькой длинны, раньшедлинные комментарии были с большим выступом, это зависело от длины текста, а сейчас расчет более честный.